ipynb notebook for calculating optimum racing lines

In [42]:
# Imports
from shapely.geometry import Point, Polygon
from shapely.geometry.polygon import LinearRing, LineString
import pandas as pd
import os.path
import copy
import geopandas as gpd
import geojsonio
import matplotlib.pyplot as plt
%matplotlib notebook
import glob
import numpy as np
from scipy.interpolate import interp1d
np.seterr(all="ignore")


{'divide': 'ignore', 'over': 'ignore', 'under': 'ignore', 'invalid': 'ignore'}

In [43]:
#install error fix
pip install geojsonio

Note: you may need to restart the kernel to use updated packages.


In [44]:
# Simulation Settings and Data Imports
interp_val = .05
iters = 500

#opening file with custom input
folder = "./circuits"
filename = ''
track = gpd.read_file(folder + '/' + filename)

#custom track input
actual_track_length = 6.003

In [ ]:
#custom track input
actual_track_length = 6.003

In [46]:
# Extract Center Line Information
trackL = track.geometry[0]
trackl

coords = np.array(trackL.coords)
y_diff_mean = np.mean(np.diff(coords[:,0]))
x_diff_mean = np.mean(np.diff(coords[:,1]))
tot_diff_mean = np.sqrt(y_diff_mean**2 + x_diff_mean**2)

#convert np array to list

y_list = [coords[0, 1]]
x_list = [coords[0, 0]]

In [47]:
# Interpolate Track and Create Width
for i in range(coords.shape[0]):
    if i > 1:
        x_i_diff = coords[i,0] - coords[i-1,0]
        y_i_diff = coords[i,1] - coords[i-1,1]
        
        tot_i_diff = np.sqrt(x_i_diff**2 + y_i_diff**2)
        
        y_list.append(coords[i,1])
        x_list.append(coords[i,0])
        
        if tot_i_diff > interp_val* tot_diff_mean:
            x_i = coords[i,0]
            x_i1 = coords[i-1,0]
            
            y_i = coords[i,1]
            y_i1 = coords[i-1, 1]
        
            x = interp1d([x_i, x_i1], [y_i, y_i1])
            
            xs = np.linspace(x_i1, x_i, 5)
            
            y = x(xs)
            
            final_x = list(xs)
            final_y = list(y)
            
            for j in range(len(final_x)):
                x_list.append(final_x[j])
                y_list.append(final_y[j])
            
    
new_track_line_data = np.zeros([len(x_list), 2])
new_track_line_data[:, 0] = x_list
new_track_line_data[:, 1] = y_list

new_track_line = LineString(new_track_line_data)

conv = actual_track_length/new_track_line.length


track_poly = new_track_line.buffer(15/conv/10000, resolution = 40)

def extract_poly_coords(geom):
    if geom.type == 'Polygon':
        exterior_coords = geom.exterior.coords[:]
        interior_coords = []
        for interior in geom.interiors:
            interior_coords += interior.coords[:]
    elif geom.type == 'MultiPolygon':
        exterior_coords = []
        interior_coords = []
        for part in geom:
            epc = extract_poly_coords(part)  # Recursive call
            exterior_coords += epc['exterior_coords']
            interior_coords += epc['interior_coords']
    else:
        raise ValueError('Unhandled geometry type: ' + repr(geom.type))
    return {'exterior_coords': exterior_coords,
            'interior_coords': interior_coords}

coord_dict = extract_poly_coords(track_poly)

center_line = new_track_line_data
inner_border = np.array(coord_dict['interior_coords'])
outer_border = np.array(coord_dict['exterior_coords'])

l_outer_border = LineString(outer_border)
l_inner_border = LineString(inner_border)
l_center_line = LineString(center_line)


road_poly = Polygon(np.vstack((l_outer_border, np.flipud(l_inner_border))))

<__array_function__ internals>:5: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
<__array_function__ internals>:5: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.


In [41]:
# start along centerline of track
race_line = copy.deepcopy(center_line[:-1])  # Use this for centerline being outer bound
for i in range(LINE_ITERATIONS):
    race_line = improve_race_line(race_line, inner_border, outer_border)
    if i % 50 == 0: print("Iteration %d" % i)
    
    
# need to put duplicate point race_line[0] at race_line[-1] to make a closed loops
loop_race_line = np.append(race_line, [race_line[0]], axis=0)

# These should be the same
print("These should be the same: ", (center_line.shape, loop_race_line.shape))
print("centerline length: %0.2f" % l_center_line.length)
print("New race line : %0.2f" % LineString(loop_race_line).length)

Iteration 0
Iteration 50
Iteration 100
Iteration 150
Iteration 200
Iteration 250
Iteration 300
Iteration 350
Iteration 400
Iteration 450
These should be the same:  ((499, 2), (499, 2))
Original centerline length: 0.17
New race line length: 0.06


In [51]:
fig = plt.figure(1, figsize=(16, 10))
ax = fig.add_subplot(111, facecolor='black')
plt.axis('equal')
print_border(ax, loop_race_line, inner_border, outer_border)


<IPython.core.display.Javascript object>

In [ ]:
html_obj = fig
# Write html object to a file (adjust file path; Windows path is used here)
with open('/Users/parthkhurana/Library/Mobile Documents/com~apple~TextEdit/Documents/display.html','wb') as f:
    f.write(html_obj.data.encode("UTF-8"))

# Open the stored HTML file on the default browser
url = r'/Users/parthkhurana/Library/Mobile Documents/com~apple~TextEdit/Documents/display.html'
webbrowser.open(url, new=2)